<a href="https://colab.research.google.com/github/SanjilMahajani/SubtitleGenerator/blob/main/Video_Subtitle_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install ffmpeg-python
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117



  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-b8xd7841
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-b8xd7841
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802824 sha256=fe8aca5999999643d31b78b60a524a542b06ed230403f1c0462dc72dbe9cc5a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-f5enpqh6/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/

In [2]:
import ffmpeg
import whisper
import torch

# Check if GPU is available and select the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the Whisper model on the selected device
model = whisper.load_model("medium", device=device)  # Use 'base', 'small', 'medium', or 'large' based on your requirement


100%|█████████████████████████████████████| 1.42G/1.42G [00:27<00:00, 55.8MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [3]:
import os
import whisper
import ffmpeg

# Check if GPU is available and select the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Function to extract audio from video
def extract_audio_from_video(video_path, audio_path="/content/audio.wav"):
    try:
        # Use ffmpeg to extract audio
        ffmpeg.input(video_path).output(audio_path).run(overwrite_output=True, quiet=True)
        print(f"Audio extracted to {audio_path}")
    except Exception as e:
        print(f"Error extracting audio: {e}")

# Function to convert seconds to SRT time format
def convert_seconds_to_srt_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    milliseconds = int((seconds % 1) * 1000)
    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

# Function to transcribe audio to English text using Whisper with timestamps
def transcribe_audio_to_english_with_timestamps(model, audio_path):
    try:
        # Transcribe audio using Whisper with translation to English and timestamps
        result = model.transcribe(audio_path, task="translate", condition_on_previous_text=False)
        segments = result['segments']
        return segments
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

# Function to create SRT file from Whisper transcription segments
def create_srt_from_segments(segments, output_srt_file):
    with open(output_srt_file, 'w') as srt_file:
        for index, segment in enumerate(segments):
            start_time = segment['start']
            end_time = segment['end']
            text = segment['text']
            start_time_str = convert_seconds_to_srt_time(start_time)
            end_time_str = convert_seconds_to_srt_time(end_time)
            srt_file.write(f"{index + 1}\n")
            srt_file.write(f"{start_time_str} --> {end_time_str}\n")
            srt_file.write(f"{text.strip()}\n\n")

    print(f"Subtitle file '{output_srt_file}' created successfully.")

# Main function to convert video to English subtitles
def video_to_english_subtitles(video_path):
    audio_path = "/content/audio.wav"
    extract_audio_from_video(video_path, audio_path)

    segments = transcribe_audio_to_english_with_timestamps(model, audio_path)
    if segments:
        create_srt_from_segments(segments, "/content/subtitles.srt")
    else:
        print("Audio transcription and translation failed.")


video_path = "/content/video.mp4"  # Replace with your video file path
video_to_english_subtitles(video_path)


Audio extracted to /content/audio.wav
Subtitle file '/content/subtitles.srt' created successfully.
